In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

In [ ]:
hex_scheduled = _hex_json.loads("false")

In [ ]:
hex_user_email = _hex_json.loads("\"example-user@example.com\"")

In [ ]:
hex_run_context = _hex_json.loads("\"logic\"")

In [ ]:
hex_timezone = _hex_json.loads("\"Asia/Dubai\"")

In [ ]:
hex_project_id = _hex_json.loads("\"c3bd2eb4-0196-4d19-b402-34c894ab6bee\"")

In [ ]:
hex_project_name = _hex_json.loads("\"COURSERA GP: Snowflake for Data Science: Intro to Snowpark ML for Python_learner Guide_Final\"")

In [ ]:
hex_status = _hex_json.loads("\"\"")

In [ ]:
hex_categories = _hex_json.loads("[]")

In [ ]:
hex_color_palette = _hex_json.loads("[\"#4C78A8\",\"#F58518\",\"#E45756\",\"#72B7B2\",\"#54A24B\",\"#EECA3B\",\"#B279A2\",\"#FF9DA6\",\"#9D755D\",\"#BAB0AC\"]")

# TASK 2 : Set-up your Snowpark for Python workspace
- In this Task, we will walk through a how to connect HEX notebook to your Snowflake Trial Account.

#### _**a-Import Libraries**_



In [ ]:
# Snowpark for Python
from snowflake.snowpark.version import VERSION
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import DecimalType

# Snowpark ML
import snowflake.ml.modeling.preprocessing as snowml
from snowflake.ml.modeling.pipeline import Pipeline

# Data Science Libs
import numpy as np
import pandas as pd

# Misc
import joblib

# warning suppresion
import warnings; warnings.simplefilter('ignore')

In [ ]:
import hextoolkit
hex_snowflake_conn = hextoolkit.get_data_connection('MY_SNOWFLAKE')
session = hex_snowflake_conn.get_snowpark_session()

#### _**b- Establish Secure Connection to Snowflake**_



In [ ]:
# Verify connectivity to Snowflake
snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

User                        : PHEONIXDEV
Role                        : "ACCOUNTADMIN"
Database                    : "DIAMONDS"
Schema                      : "PUBLIC"
Warehouse                   : "PC_HEX_WH"
Snowflake version           : 7.41.0
Snowpark for Python version : 1.6.1


# TASK 3 : Data Preprocessing: Transform categorical variables
- In this Task, we will walk through data transformations that are included in the Snowpark ML Preprocessing API.

*We will illustrate a few transformation functions availabe in Snowpark ML, the rest can be found in the [documentation](https://docs.snowflake.com/LIMITEDACCESS/snowflake-ml-preprocessing).*

#### _a- Data Loading_



In [ ]:
# Specify the table name where we stored the diamonds dataset
# ** ONLY Change this only if you named your table something else in the data ingest notebook **
DIAMONDS_TABLE = 'diamonds'
input_tbl = f"{session.get_current_database()}.{session.get_current_schema()}.{DIAMONDS_TABLE}"

In [ ]:
# First, we read-in the data from a Snowflake table into a Snowpark DataFrame
diamonds_df = session.table(input_tbl)

# Let's visualise the Data
diamonds_df.show()

# Describe Snowpark Datafarame
diamonds_df.describe()

-------------------------------------------------------------------------------------------------
|"CARAT"  |"COLOR"  |"CLARITY"  |"TABLE_PCT"  |"PRICE"  |"X"  |"Y"  |"Z"  |"CUT"      |"DEPTH"  |
-------------------------------------------------------------------------------------------------
|0        |E        |SI2        |55           |326      |4    |4    |2    |IDEAL      |62       |
|0        |E        |SI1        |61           |326      |4    |4    |2    |PREMIUM    |60       |
|0        |E        |VS1        |65           |327      |4    |4    |2    |GOOD       |57       |
|0        |I        |VS2        |58           |334      |4    |4    |3    |PREMIUM    |62       |
|0        |J        |SI2        |58           |335      |4    |4    |3    |GOOD       |63       |
|0        |J        |VVS2       |57           |336      |4    |4    |2    |VERY_GOOD  |63       |
|0        |I        |VVS1       |57           |336      |4    |4    |2    |VERY_GOOD  |62       |
|0        |H        

In [ ]:
# Convert Snowpark DF API into Pandas DF, then check for missing values and categorical variables in the dataset
diamonds_df.to_pandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53920 entries, 0 to 53919
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   CARAT      53920 non-null  int8  
 1   COLOR      53920 non-null  object
 2   CLARITY    53920 non-null  object
 3   TABLE_PCT  53920 non-null  int8  
 4   PRICE      53920 non-null  int16 
 5   X          53920 non-null  int8  
 6   Y          53920 non-null  int8  
 7   Z          53920 non-null  int8  
 8   CUT        53920 non-null  object
 9   DEPTH      53920 non-null  int8  
dtypes: int16(1), int8(6), object(3)
memory usage: 1.6+ MB


<img src="https://4cs.gia.edu/wp-content/uploads/2012/05/Cut-Anatomy-of-Diamond.png" style="height:200px" />

_**Features**_

- **Price :**, in US dollars ($326--$18,823)This is the target column containing tags for the features. 

The 4 Cs of Diamonds:

- **Carat**, ,**(0.2--5.01)**, The carat is the diamond’s physical weight measured in metric carats.  One carat equals 1/5 gram and is subdivided into 100 points. Carat weight is the most objective grade of the 4Cs. 
- _**Cut (Fair, Good, Very Good, Premium, Ideal):**_, The quality of the cut. The more precise the diamond is cut, the more captivating the diamond is to the eye thus of high grade.
- _**Clarity (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best)):**_, Diamonds can have internal characteristics known as inclusions or external characteristics known as blemishes. Diamonds without inclusions or blemishes are rare; however, most characteristics can only be seen with magnification.
- _**Color (from J (worst) to D (best)): **_,The color of gem-quality diamonds occurs in many hues. In the range from colorless to light yellow or light brown. Colorless diamonds are the rarest. Other natural colors (blue, red, pink for example) are known as "fancy,” and their color grading is different from white colorless diamonds.
- Dimensions: 
  - **X: **,length in mm (0--10.74)
  - **Y: **,width in mm (0--58.9)
  - **Z: **,depth in mm (0--31.8)



#### b-Transform `COLOR`, `CLARITY` and `CUT` from categorical to numerical values using `OrdinalEncoder`



In [ ]:
import snowflake.ml.modeling.preprocessing as snowml

# Encode CUT, CLARITY, and COLOR to preserve ordinal importance
categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
    "COLOR": np.array(['D', 'E', 'F', 'G', 'H', 'I', 'J']),
}
snowml_oe = snowml.OrdinalEncoder(input_cols=["CUT", "CLARITY", "COLOR"], 
                                    output_cols=["CUT_OE", "CLARITY_OE","COLOR_OE"], 
                                    categories=categories
                                )

ord_encoded_diamonds_df = snowml_oe.fit(diamonds_df).transform(diamonds_df)

# Show the encoding
print(snowml_oe._state_pandas)

   _COLUMN_NAME  _CATEGORY  _INDEX
0           CUT      IDEAL       0
1           CUT    PREMIUM       1
2           CUT  VERY_GOOD       2
3           CUT       GOOD       3
4           CUT       FAIR       4
5       CLARITY         IF       0
6       CLARITY       VVS1       1
7       CLARITY       VVS2       2
8       CLARITY        VS1       3
9       CLARITY        VS2       4
10      CLARITY        SI1       5
11      CLARITY        SI2       6
12      CLARITY         I1       7
13      CLARITY         I2       8
14      CLARITY         I3       9
15        COLOR          D       0
16        COLOR          E       1
17        COLOR          F       2
18        COLOR          G       3
19        COLOR          H       4
20        COLOR          I       5
21        COLOR          J       6


In [ ]:
#show the results of the OrdinalEncoder transformer
ord_encoded_diamonds_df.show()

----------------------------------------------------------------------------------------------------------------------------------------
|"CUT_OE"  |"CLARITY_OE"  |"COLOR_OE"  |"CARAT"  |"COLOR"  |"CLARITY"  |"TABLE_PCT"  |"PRICE"  |"X"  |"Y"  |"Z"  |"CUT"      |"DEPTH"  |
----------------------------------------------------------------------------------------------------------------------------------------
|0.0       |6.0           |1.0         |0        |E        |SI2        |55           |326      |4    |4    |2    |IDEAL      |62       |
|1.0       |5.0           |1.0         |0        |E        |SI1        |61           |326      |4    |4    |2    |PREMIUM    |60       |
|3.0       |3.0           |1.0         |0        |E        |VS1        |65           |327      |4    |4    |2    |GOOD       |57       |
|1.0       |4.0           |5.0         |0        |I        |VS2        |58           |334      |4    |4    |3    |PREMIUM    |62       |
|3.0       |6.0           |6.0         |0

# PRACTICE TASK : Normalise Numerical Column `CARAT` using `MinMaxScaler`

SNOWPARK ML Modeling preprocessing [Documentation](https://docs.snowflake.com/developer-guide/snowpark-ml/snowpark-ml-modeling)



In [ ]:
from snowflake.ml.modeling.preprocessing import MinMaxScaler# >Your script goes here...<
# hint : to get help run: 
# help(MinMaxScaler)

In [ ]:
#MinMaxScaler scales the data to a fixed range, typically between 0 and 1

# Normalize the CARAT column
snowml_mms = snowml.MinMaxScaler(input_cols=["CARAT"], output_cols=["CARAT_NORM"])

# Call : 
# a-fit(X[, y]) Compute the minimum and maximum to be used for later scaling.
# b-transform(X) Scale features of X according to feature_range.

normalized_diamonds_df = snowml_mms.fit(diamonds_df).transform(diamonds_df)

# Check Results : Call .Describe() then .Show() Results
normalized_diamonds_df.describe().show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"CARAT_NORM"         |"CARAT"             |"COLOR"  |"CLARITY"  |"TABLE_PCT"        |"PRICE"             |"X"                 |"Y"                 |"Z"                 |"CUT"      |"DEPTH"             |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |53920.0              |53920.0             |53920    |53920      |53920.0            |53920.0             |53920.0             |53920.0             |53920.0             |53920      |53920.0             |
|mean       |0.1581120178041543   |0.79056             |NULL     |NULL       |57.457808          |3930.993231         |5.69757          

# TASK 4 : Build Preprocessing Pipeline
- In this Task, we will build a preprocessing pipeline to be used for both the ML training & inference steps to have standarized feature transformations.

#### _a- Categorise all the features for processing_



In [ ]:
# Categorize all the features for processing
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"] # To name the ordinal encoded columns

categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
    "COLOR": np.array(['D', 'E', 'F', 'G', 'H', 'I', 'J']),
}

#### _b- Build the pipeline_



In [ ]:
# Build the pipeline
preprocessing_pipeline = Pipeline(
    steps=[
            (
                "OE",
                snowml.OrdinalEncoder(
                    input_cols=CATEGORICAL_COLUMNS,
                    output_cols=CATEGORICAL_COLUMNS_OE,
                    categories=categories,
                )
            )
    ]
)

PIPELINE_FILE = 'preprocessing_pipeline.joblib'

# Lets create joblib file of the pipeline and save it locally
joblib.dump(preprocessing_pipeline,PIPELINE_FILE)

['preprocessing_pipeline.joblib']

#### _c- Save the pipleine into Snowflake stage as a joblib file_



In [ ]:
# You can also save the pipeline joblibfile file into a Snowflake Internal Stage
session.file.put(PIPELINE_FILE, "ML_FILES",overwrite=True)


[PutResult(source='preprocessing_pipeline.joblib', target='preprocessing_pipeline.joblib.gz', source_size=1714, target_size=960, source_compression='NONE', target_compression='GZIP', status='UPLOADED', message='')]

# TASK 5 : Build an `XGBoost` Regression Model
- In this Task, we will illustrate how to train an XGBoost model with the diamonds dataset using the Snowpark ML Model API. 

The Snowpark ML Model API currently supports sklearn, xgboost, and lightgbm models, for more details check out Snowflake Documentation : https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-modeling 

#### _a- Import Libraries_



In [ ]:
# Snowpark ML
from snowflake.ml.modeling.xgboost import XGBRegressor
# For All Snowpark ML Modeling Classes visit https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-modeling

# joblib
import joblib

#### _b- Load the data & preprocessing pipeline_



In [ ]:
# Load Diamonds Table from Snowflake
# diamonds_df = session.table(input_tbl)
diamonds_df.show()

-------------------------------------------------------------------------------------------------
|"CARAT"  |"COLOR"  |"CLARITY"  |"TABLE_PCT"  |"PRICE"  |"X"  |"Y"  |"Z"  |"CUT"      |"DEPTH"  |
-------------------------------------------------------------------------------------------------
|0        |E        |SI2        |55           |326      |4    |4    |2    |IDEAL      |62       |
|0        |E        |SI1        |61           |326      |4    |4    |2    |PREMIUM    |60       |
|0        |E        |VS1        |65           |327      |4    |4    |2    |GOOD       |57       |
|0        |I        |VS2        |58           |334      |4    |4    |3    |PREMIUM    |62       |
|0        |J        |SI2        |58           |335      |4    |4    |3    |GOOD       |63       |
|0        |J        |VVS2       |57           |336      |4    |4    |2    |VERY_GOOD  |63       |
|0        |I        |VVS1       |57           |336      |4    |4    |2    |VERY_GOOD  |62       |
|0        |H        

In [ ]:
# Load the preprocessing pipeline object
#preprocessing_pipeline = joblib.load('preprocessing_pipeline.joblib')

# Visualise the Pipeline 
preprocessing_pipeline.to_sklearn()

Pipeline(steps=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}))]) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. Pipeline Pipeline(steps=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}))]) OrdinalEncoder OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')})

In [ ]:
# Categorize all the features for modeling
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"] 
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

LABEL_COLUMNS = ['PRICE']
OUTPUT_COLUMNS = ['PREDICTED_PRICE']

#### _c- Build XGBoost Regression Model_



In [ ]:
# Split the data into train and test sets
diamonds_train_df, diamonds_test_df = diamonds_df.random_split(weights=[0.9,0.1], seed=0)

# Run the train and test sets through the Pipeline object we defined earlier
train_df = preprocessing_pipeline.fit(diamonds_train_df).transform(diamonds_train_df)
test_df = preprocessing_pipeline.transform(diamonds_test_df)

In [ ]:
# Define the XGBRegressor
regressor = XGBRegressor(
    input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
)

# Train
regressor.fit(train_df)

# Predict
result = regressor.predict(test_df)

#### _d- Analyse Model Accuracy using Snowpark ML's R-2 Score._



In [ ]:
from snowflake.ml.modeling.metrics import r2_score

# Show Result
result.select("PRICE", "PREDICTED_PRICE").show()

# Model Accuracy using R-2 Score
print('Acccuracy:', r2_score(df=result,y_true_col_name="PRICE",y_pred_col_name="PREDICTED_PRICE"))

--------------------------------
|"PRICE"  |"PREDICTED_PRICE"   |
--------------------------------
|351      |331.02655029296875  |
|353      |290.8585510253906   |
|355      |466.9120178222656   |
|357      |463.9052734375      |
|554      |615.9501342773438   |
|554      |544.3433227539062   |
|2757     |4150.7080078125     |
|2759     |4315.59423828125    |
|2759     |3629.06494140625    |
|2762     |3550.974853515625   |
--------------------------------

Acccuracy: 0.9348540979849457


#### _e- Plot Actual Vs Predicted _



In [ ]:
import altair
chart_result = altair.Chart.from_json("""
{
    "width": "container",
    "height": "container",
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "layer": [
        {
            "resolve": {
                "scale": {
                    "color": "independent",
                    "y": "shared"
                }
            },
            "layer": [
                {
                    "transform": [
                        {
                            "filter": "isValid(datum[\"PRICE\"])"
                        }
                    ],
                    "layer": [
                        {
                            "mark": {
                                "type": "point",
                                "filled": true,
                                "clip": true,
                                "cursor": "crosshair"
                            }
                        }
                    ],
                    "encoding": {
                        "x": {
                            "field": "PRICE",
                            "type": "quantitative",
                            "title": "PRICE",
                            "scale": {
                                "padding": 8
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "y": {
                            "field": "PREDICTED_PRICE",
                            "type": "quantitative",
                            "title": "PREDICTED_PRICE",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "value": "#4C78A8"
                        },
                        "opacity": {
                            "value": 1
                        }
                    }
                },
                {
                    "mark": {
                        "type": "point",
                        "filled": true,
                        "clip": true,
                        "cursor": "crosshair"
                    },
                    "encoding": {
                        "x": {
                            "field": "PRICE",
                            "type": "quantitative",
                            "title": "PRICE",
                            "scale": {
                                "padding": 8
                            },
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "y": {
                            "field": "PREDICTED_PRICE",
                            "type": "quantitative",
                            "title": "PREDICTED_PRICE",
                            "scale": {},
                            "axis": {
                                "grid": true,
                                "ticks": true,
                                "labels": true,
                                "labelFlush": false,
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            }
                        },
                        "color": {
                            "value": "#4C78A8"
                        },
                        "tooltip": [
                            {
                                "field": "PRICE",
                                "type": "quantitative",
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER"
                            },
                            {
                                "field": "PREDICTED_PRICE",
                                "type": "quantitative",
                                "format": {
                                    "format": "NUMBER",
                                    "columnType": "NUMBER",
                                    "numDecimalDigits": -1,
                                    "currency": "$",
                                    "nanFormat": ""
                                },
                                "formatType": "NUMBER_FORMATTER",
                                "title": "PREDICTED_PRICE"
                            }
                        ],
                        "opacity": {
                            "value": 0,
                            "condition": {
                                "param": "point_hover_d41564d2-fcf5-43b1-9cf0-294ecafeaae1",
                                "value": 1,
                                "empty": false
                            }
                        },
                        "size": {
                            "value": 100
                        }
                    },
                    "params": [
                        {
                            "name": "point_hover_d41564d2-fcf5-43b1-9cf0-294ecafeaae1",
                            "select": {
                                "type": "point",
                                "on": "mouseover",
                                "nearest": false
                            }
                        }
                    ]
                }
            ]
        }
    ],
    "config": {
        "legend": {
            "orient": "right"
        },
        "font": "\"IBM Plex Sans\", system-ui, -apple-system, BlinkMacSystemFont, sans-serif",
        "view": {}
    },
    "datasets": {
        "layer00": [
            {
                "name": "dummy",
                "value": 0
            }
        ]
    },
    "usermeta": {
        "selectionConfigs": {}
    }
}
""")
chart_result.datasets.layer00 = result.to_json(orient='records')
chart_result.display(actions=False)

# PRACTICE TASK : Try different Regression Model `LinearRegression`  

In [ ]:
# Import LinearRegression 
from snowflake.ml.modeling.linear_model import LinearRegression

In [ ]:
# Define the LinearRegression
lr= LinearRegression(
    input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
)

# Train
lr.fit(train_df)

# Predict
lr_result = lr.predict(test_df)

**-->** Navigate to Snowflake Query History to check how Snowflake translates and executes your Python queries 



In [ ]:
# Show Result   
lr_result.select("PRICE", "PREDICTED_PRICE").show()

# Model Accuracy using R-2 Score
print('Acccuracy:', r2_score(df=lr_result,y_true_col_name="PRICE",y_pred_col_name="PREDICTED_PRICE"))


---------------------------------
|"PRICE"  |"PREDICTED_PRICE"    |
---------------------------------
|351      |-2344.0882349605854  |
|353      |-2213.5540001648305  |
|355      |-2990.9063607836943  |
|357      |-2494.160568881115   |
|554      |-1780.045246159836   |
|554      |-2212.0016637624303  |
|2757     |5471.961862968939    |
|2759     |5619.849950688946    |
|2759     |4069.9758249393344   |
|2762     |5360.939114755582    |
---------------------------------

Acccuracy: 0.8052799514152995


# TASK 6: Fine Tune Model's Hyperparameters using `GridSearchCV()`

In [ ]:
#Import Libraries
from snowflake.ml.modeling.model_selection import GridSearchCV
#help(GridSearchCV)

#### _a- Use GridSearchCV to find the best fitting model_



In [ ]:
#from sklearn.metrics import get_scorer_names
#get_scorer_names()

In [ ]:
# Documentation : https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

grid_search = GridSearchCV(
    estimator=regressor, # same XGBOOST model which we trained earlier
    param_grid={
        "n_estimators":[100,200,300, 400, 500], # Number of boosting rounds
        "learning_rate":[0.1,0.2,0.3, 0.4, 0.5], # Step size shrinkage to prevent overfitting
    },
    n_jobs = -1, #Number of jobs to run in parallel. -1 means using all processors.
    scoring="r2",
    cv=5, # 5-fold cross-validation
    input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
)

# Train hypertuned model 
grid_search.fit(train_df)

> **NOTE**>  : Process will take ~2.5min to run, in order to accelerate it, increase the size of your Snowflake virtual Warehouse.



In [ ]:
# Best Estimator (model) Hyperparameters
grid_search.to_sklearn().best_params_

{'learning_rate': 0.1, 'n_estimators': 200}

> **NOTE**>  : Based on our Grid search results above the best estimator with the lowest R-2 Score hyper-parameters are learning_rate=0.1 and n_estimator=200.



#### b- Calculate R-2 Score based on best estimator 



In [ ]:
# Predict Price using Test set using best estimator (model)
result_optimal_model = grid_search.predict(test_df)

# Analyze results using R-2 Score 
print('Acccuracy of Hypertuned XGBOOST Model:', r2_score(df=result_optimal_model,y_true_col_name="PRICE",y_pred_col_name="PREDICTED_PRICE"))
print('Acccuracy of Initial XGBOOST Model:', r2_score(df=result,y_true_col_name="PRICE",y_pred_col_name="PREDICTED_PRICE"))

Acccuracy of Hypertuned XGBOOST Model: 0.935681087990025
Acccuracy of Initial XGBOOST Model: 0.9348540979849457


# TASK 7: Deploy the model as a Vectorized User-Defined Function (UDF)

> ___**Note:**___> _ When you call `model.predict()` function Snowpark ML creates a temporary UDF, so in order to persist as a permanent UDF, we'll need to define our own UDF.  -- Navigate to Snowflake Query History to see executed queries -- _



In [ ]:
#import libraries 
import joblib
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import OrdinalEncoder
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.snowpark import types as T
import snowflake.snowpark.functions as F
import cachetools
import pandas as pd

#### _a- Create the end to end pipeline_



In [ ]:
# Categorize all the features for modeling
CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
CATEGORICAL_COLUMNS_OE = ["CUT_OE", "COLOR_OE", "CLARITY_OE"]
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

categories = {
    "CUT": np.array(["IDEAL", "PREMIUM", "VERY_GOOD", "GOOD", "FAIR"]),
    "CLARITY": np.array(["IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"]),
    "COLOR": np.array(['D', 'E', 'F', 'G', 'H', 'I', 'J']),
}

# Define the end to end pipeline that contains Transforms (preprocessing) and Estimator (optimal XGBRegressor model)
pipe = Pipeline(
    steps=[
        ("OE", OrdinalEncoder(
            input_cols= CATEGORICAL_COLUMNS,  
            output_cols= CATEGORICAL_COLUMNS_OE, 
            categories = categories,  
            drop_input_cols=True)
            ),
        ("regressor", XGBRegressor(
            learning_rate = 0.1, # Add Best best_params_ Results here 
            n_estimators = 200,  # Add Best best_params_ Results here 
            input_cols=CATEGORICAL_COLUMNS_OE + NUMERICAL_COLUMNS, 
            label_cols=["PRICE"], 
            output_cols=['PREDICTED_PRICE'], 
            n_jobs=-1)
            )
    ]
)

# Random split
diamonds_train_df, diamonds_test_df = diamonds_df.random_split(weights=[0.9,0.1], seed =0)

# Train the model and convert it to sklearn pipeline
xgb_optimal_model = pipe.fit(diamonds_train_df).to_sklearn()

#### b- Save the optimal pipeline within a Snowflake Internal Stage



In [ ]:
from sklearn import set_config
set_config(display="diagram")
xgb_optimal_model

Pipeline(steps=[('OE',
 ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 [7, 1, 2])])),
 ('filter...
 feature_types=None, gamma=None, gpu_id=None,
 grow_policy=None, importance_type=None,
 interaction_constraints=None, learning_rate=0.1,
 max_bin=None, max_cat_threshold=None,
 max_cat_to_onehot=None, max_delta_step=None,
 max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan,
 monotone_constraints=None, n_estimators=200,
 n_jobs=-1, num_parallel_tree=None, predictor=None,
 random_state=None, ...))]) In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org. Pipeline Pipeline(steps=[('OE',
 ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 [7, 1, 2])])),
 ('filter...
 feature_types=None, gamma=None, gpu_id=None,
 grow_policy=None, importance_type=None,
 interaction_constraints=None, learning_rate=0.1,
 max_bin=None, max_cat_threshold=None,
 max_cat_to_onehot=None, max_delta_step=None,
 max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan,
 monotone_constraints=None, n_estimators=200,
 n_jobs=-1, num_parallel_tree=None, predictor=None,
 random_state=None, ...))]) OE: ColumnTransformer ColumnTransformer(remainder='passthrough',
 transformers=[('OE',
 OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}),
 [7, 1, 2])]) OE [7, 1, 2] OrdinalEncoder OrdinalEncoder(categories={'CLARITY': array(['IF', 'VVS1', 'VVS2', 'VS1', 'VS2', 'SI1', 'SI2', 'I1', 'I2', 'I3'],
 dtype='<U4'),
 'COLOR': array(['D', 'E', 'F', 'G', 'H', 'I', 'J'], dtype='<U1'),
 'CUT': array(['IDEAL', 'PREMIUM', 'VERY_GOOD', 'GOOD', 'FAIR'], dtype='<U9')}) remainder ['CARAT', 'TABLE_PCT', 'X', 'Y', 'Z', 'DEPTH'] passthrough passthrough filter_input_cols_for_estimator: ColumnTransformer ColumnTransformer(remainder='passthrough',
 transformers=[('filter_input_cols_for_estimator',
 'passthrough', [0, 1, 2, 3, 8, 4, 5, 6, 7])]) filter_input_cols_for_estimator [0, 1, 2, 3, 8, 4, 5, 6, 7] passthrough passthrough remainder [] passthrough passthrough XGBRegressor XGBRegressor(base_score=None, booster=None, callbacks=None,
 colsample_bylevel=None, colsample_bynode=None,
 colsample_bytree=None, early_stopping_rounds=None,
 enable_categorical=False, eval_metric=None, feature_types=None,
 gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
 interaction_constraints=None, learning_rate=0.1, max_bin=None,
 max_cat_threshold=None, max_cat_to_onehot=None,
 max_delta_step=None, max_depth=None, max_leaves=None,
 min_child_weight=None, missing=nan, monotone_constraints=None,
 n_estimators=200, n_jobs=-1, num_parallel_tree=None,
 predictor=None, random_state=None, ...)

In [ ]:
# Define a file name for the model
MODEL_FILE = 'xgb_optimal_pipe.joblib'

# Serialize and save the model to a file
joblib.dump(xgb_optimal_model, MODEL_FILE)

# Upload the model file to the specified stage
session.file.put(MODEL_FILE, "@ML_FILES", overwrite=True, auto_compress=False)

[PutResult(source='xgb_optimal_pipe.joblib', target='xgb_optimal_pipe.joblib', source_size=756422, target_size=756432, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

#### c- Register the model as a Vectorized UDF



_Vectorized Python UDFs let you define Python functions that receive batches of input rows as Pandas DataFrames and return batches of results as Pandas arrays or Series. You call vectorized Python UDFs the same way you call other Python UDFs.
Advantages of using vectorized Python UDFs compared to the default row-by-row processing pattern include:
_

- _The potential for better performance if your Python code operates efficiently on batches of rows._
- _Less transformation logic is required if you are calling into libraries that operate on Pandas DataFrames or Pandas arrays._

_For more details please visit _[Snowflake Documentation](https://docs.snowflake.com/en/developer-guide/udf/python/udf-python-batch)_._



In [ ]:
# Define a function to read the model from a file
@cachetools.cached(cache={})
def read_file(filename):
    import joblib
    import sys
    import os

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = joblib.load(file)
            return m


# Create a vectorized UDF for batch inference
@F.udf(name="predict_diamond_price",
        is_permanent=True,
        stage_location = '@ML_FILES',
        imports=['@ML_FILES/xgb_optimal_pipe.joblib'],
        packages=['snowflake-ml-python', 'joblib', 'scikit-learn==1.2.2', 'xgboost==1.7.3', 'cachetools'],
        replace=True,
        session=session)
def predict_diamond_price(pd_input: T.PandasDataFrame[str, str, str, float, float, float, float, float, float]) -> T.PandasSeries[float]:
        # Make sure you have the columns in the expected order in the Pandas DataFrame
    features = ["CUT", "COLOR", "CLARITY", "CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]
    pd_input.columns =  features
    model =  read_file('xgb_optimal_pipe.joblib')           
    prediction =   model.predict(pd_input)     
    return prediction         

#### d-Call the UDF and Predict Price for New Diamonds Table



In [ ]:
# Create a Snowpark DF containing Diamonds_NEW Table
new_diamonds = session.table('DIAMONDS_NEW')

# Apply the UDF to your DataFrame
new_diamonds_w_prediction = new_diamonds.with_column("PREDICTED_PRICE", F.call_function("predict_diamond_price", 
                                    F.col("CUT"), F.col("COLOR"), F.col("CLARITY"), 
                                    F.col("CARAT"), F.col("DEPTH"), F.col("TABLE_PCT"), 
                                    F.col("X"), F.col("Y"), F.col("Z"))
                                    )

# Show the result
new_diamonds_w_prediction.show()

------------------------------------------------------------------------------------------------------------
|"CARAT"  |"COLOR"  |"CLARITY"  |"TABLE_PCT"  |"X"  |"Y"  |"Z"  |"CUT"      |"DEPTH"  |"PREDICTED_PRICE"   |
------------------------------------------------------------------------------------------------------------
|1        |H        |SI2        |57           |7    |7    |4    |IDEAL      |61       |4968.32421875       |
|1        |E        |VVS1       |60           |7    |7    |4    |PREMIUM    |61       |12818.06640625      |
|1        |J        |SI1        |56           |7    |7    |4    |PREMIUM    |62       |4807.0283203125     |
|0        |E        |VVS1       |58           |4    |4    |2    |PREMIUM    |61       |651.5300903320312   |
|1        |H        |VS1        |57           |6    |6    |4    |PREMIUM    |62       |3787.286865234375   |
|0        |I        |SI1        |59           |5    |5    |3    |PREMIUM    |60       |670.1054077148438   |
|1        |I       

#### e-Save Predictions in a  new Snowflake Table 



In [ ]:
# Write predictions to a Snowflake table
new_diamonds_w_prediction.write.mode('overwrite').save_as_table('new_diamonds_w_prediction')

In [ ]:
#session.close()

# ------- END OF THE GUIDED PROJECT.... CONGRATULATIONS !! -------

